## TorchScript 

### JIT Tracing 

In [1]:
import torch
import torchvision

In [4]:
model = torchvision.models.alexnet(weights=None)
# trace_model is now a GRAPH, not EAGER
model.eval()
example_input = torch.rand(1,3,224,224)
traced_model = torch.jit.trace(model, example_input)


In [5]:
# Save the traced_model
traced_model.save("alexnet_traced.pt")

In [7]:
print(traced_model)

AlexNet(
  original_name=AlexNet
  (features): Sequential(
    original_name=Sequential
    (0): Conv2d(original_name=Conv2d)
    (1): ReLU(original_name=ReLU)
    (2): MaxPool2d(original_name=MaxPool2d)
    (3): Conv2d(original_name=Conv2d)
    (4): ReLU(original_name=ReLU)
    (5): MaxPool2d(original_name=MaxPool2d)
    (6): Conv2d(original_name=Conv2d)
    (7): ReLU(original_name=ReLU)
    (8): Conv2d(original_name=Conv2d)
    (9): ReLU(original_name=ReLU)
    (10): Conv2d(original_name=Conv2d)
    (11): ReLU(original_name=ReLU)
    (12): MaxPool2d(original_name=MaxPool2d)
  )
  (avgpool): AdaptiveAvgPool2d(original_name=AdaptiveAvgPool2d)
  (classifier): Sequential(
    original_name=Sequential
    (0): Dropout(original_name=Dropout)
    (1): Linear(original_name=Linear)
    (2): ReLU(original_name=ReLU)
    (3): Dropout(original_name=Dropout)
    (4): Linear(original_name=Linear)
    (5): ReLU(original_name=ReLU)
    (6): Linear(original_name=Linear)
  )
)


In [8]:
print(traced_model.code)

def forward(self,
    x: Tensor) -> Tensor:
  classifier = self.classifier
  avgpool = self.avgpool
  features = self.features
  _0 = (avgpool).forward((features).forward(x, ), )
  input = torch.flatten(_0, 1)
  return (classifier).forward(input, )



In [6]:
# Load model and run it
loaded_model = torch.jit.load("alexnet_traced.pt")
out = loaded_model(torch.rand(1,3,224,224))
print(out.shape)

torch.Size([1, 1000])
